In [10]:
from qiskit import QuantumCircuit
import numpy as np
from qiskit_aer.primitives import SamplerV2

from qbench.schemes.postselection import benchmark_discrimination_using_postselection
from qbench.schemes.direct_sum import benchmark_discrimination_using_direct_sum

In [11]:
from qiskit_aer import StatevectorSimulator

simulator = StatevectorSimulator()

In [12]:
### discrimination experiment for measurement in Hadamard basis using postselection and direct sum###

def state_prep():
    circuit = QuantumCircuit(2)
    circuit.h(0)
    circuit.cx(0, 1)
    return circuit.to_instruction()

def u_dag():
    circuit = QuantumCircuit(1)
    circuit.h(0)
    return circuit.to_instruction()

def v0_dag():
    circuit = QuantumCircuit(1)
    circuit.ry(-np.pi * 3 / 4, 0)
    return circuit.to_instruction()
    
def v1_dag():
    circuit = QuantumCircuit(1)
    circuit.ry(-np.pi * 3 / 4, 0)
    circuit.x(0)
    return circuit.to_instruction()
    
def v0_v1_direct_sum_dag():
    circuit = QuantumCircuit(2)
    circuit.ry(-np.pi * 3 / 4, 0)
    circuit.cx(0, 1)
    return circuit.to_instruction()

In [13]:
discrimination_postselection_result = benchmark_discrimination_using_postselection(
    backend=simulator,
    target=0,
    ancilla=1,
    state_preparation=state_prep(),
    u_dag=u_dag(),
    v0_dag=v0_dag(),
    v1_dag=v1_dag(),
    num_shots_per_measurement=100000,)

In [14]:
discrimination_direct_sum_result = benchmark_discrimination_using_direct_sum(
    backend=simulator,
    target=1,
    ancilla=2,
    state_preparation=state_prep(),
    u_dag=u_dag(),
    v0_v1_direct_sum_dag=v0_v1_direct_sum_dag(),
    num_shots_per_measurement=100000,)

In [15]:
p_succ = (2 + np.sqrt(2)) / 4
print(f"Analytical p_succ = {p_succ}")
print(f"Postselection: p_succ = {discrimination_postselection_result}, abs.error ={np.abs(p_succ - discrimination_postselection_result)}")
print(f"Direct sum: p_succ = {discrimination_direct_sum_result}, abs.error ={np.abs(p_succ - discrimination_direct_sum_result)}")

Analytical p_succ = 0.8535533905932737
Postselection: p_succ = 0.8549341565062727, abs.error =0.0013807659129989602
Direct sum: p_succ = 0.852805, abs.error =0.0007483905932736956


In [16]:
from qbench.schemes.postselection import (
    assemble_circuits_discrimination_postselection,
    compute_probabilities_discrimination_postselection,)

circuits = assemble_circuits_discrimination_postselection(
    target=0,
    ancilla=1,
    state_preparation=state_prep(),
    u_dag=u_dag(),
    v0_dag=v0_dag(),
    v1_dag=v1_dag(),)

In [17]:
from qiskit_aer.noise import NoiseModel, ReadoutError

error = ReadoutError([[0.75, 0.25], [0.8, 0.2]])
noise_model = NoiseModel()
noise_model.add_readout_error(error, [0])
noise_model.add_readout_error(error, [1])


In [18]:
keys_ordering = ["id_v0", "id_v1", "u_v0", "u_v1"]

all_circuits = [circuits[key] for key in keys_ordering]

sampler_noiseless = SamplerV2()
sampler_noisy = SamplerV2(options=dict(backend_options=dict(noise_model=noise_model)))

counts_noisy = [sampler_noisy.run(
    [circ],
    shots=10000).result()[0].data.meas.get_counts() for circ in all_circuits]

counts_noiseless = [sampler_noiseless.run(
    [circ],
    shots=10000).result()[0].data.meas.get_counts() for circ in all_circuits]

In [19]:
prob_succ_noiseless = compute_probabilities_discrimination_postselection(
    id_v0_counts=counts_noiseless[0],
    id_v1_counts=counts_noiseless[1],
    u_v0_counts=counts_noiseless[2],
    u_v1_counts=counts_noiseless[3],)

prob_succ_noisy = compute_probabilities_discrimination_postselection(
    id_v0_counts=counts_noisy[0],
    id_v1_counts=counts_noisy[1],
    u_v0_counts=counts_noisy[2],
    u_v1_counts=counts_noisy[3],)

In [20]:
p_succ = (2 + np.sqrt(2)) / 4
print(f"Analytical p_succ = {p_succ}")
print(f"Prob_succ_noiseless {prob_succ_noiseless}, abs.error ={np.abs(p_succ - prob_succ_noiseless)}")
print(f"Prob_succ_noisy = {prob_succ_noisy}, abs.error ={np.abs(p_succ - prob_succ_noisy)}")

Analytical p_succ = 0.8535533905932737
Prob_succ_noiseless 0.8572140850212383, abs.error =0.0036606944279645726
Prob_succ_noisy = 0.5044143159173481, abs.error =0.34913907467592564
